In [1]:
import time
st = time.time()
df = pd.read_csv('temp_bq_data/hackathon_data.csv')
df.head()
print("took", time.time()-st)
#df.info()

,customer_id,manufacturer_id,society_id,city_id,route_id,store_id,order_id,order_date,category_id,subcategory_id,product_id,product_quantity,selling_price_per_unit,total_cost,subscription,product_addedtobasket_on
0,2698080,1122016,1127168,1120112,1121456,1120112,338048928,2018-05-01,1122576,1125264,1120336,4,16.01,64.04,0,2018-04-30 16:23:46
1,1134224,1134336,1120224,1120112,1120336,1120112,337533168,2018-05-01,1122576,1125152,1939280,1,8.79,8.79,1,2018-04-19 01:30:02
2,3686704,1150128,1126160,1120112,1123472,1120112,338235520,2018-05-01,1123472,1130640,1681456,1,25.00,25.00,0,2018-04-30 22:26:07
3,4045328,1122016,1137360,1120112,1121456,1120112,337818768,2018-05-01,1122576,1125264,1120224,3,20.38,61.14,1,2018-04-23 01:30:03
4,3568656,1122016,1120784,1120112,1122576,1120112,338234400,2018-05-01,1122576,1125264,1120112,3,25.38,76.14,0,2018-04-30 22:22:57


took 17.913174152374268


In [2]:
def pre_process(cust_id):

    threshold = 5
    df_new = df[df['customer_id']==cust_id]
    df_new = df_new[df_new['product_quantity']>=threshold]

    return df_new

def get_suggestions(cust_id): 
    df_new = pre_process(cust_id)
    df_grouped = df_new.groupby(['product_id', 'order_date']).sum()
    # df_grouped.info()
    df_grouped = df_grouped[['product_quantity', 'subscription']]
    # df_grouped.head()

    i=0
    all_tuples = list()
    for tup in df_grouped.itertuples():
        all_tuples.append(tuple(tup))

    # all_tuples[:3]
    if len(all_tuples) == 0:
        return ['a'], ['a']
    (pre_pro_id, pre_date), pre_q, sub = all_tuples[0]
    all_days = list()
    temp = list()
    for loc in range(1, len(all_tuples)):
        (pro_id, date), q, sub = all_tuples[loc]
        if  pre_pro_id == pro_id:
            pre_date = pre_date.split('-')
            date = date.split('-')
            days = (int(date[0])*365 + int(date[1])*30 + int(date[2])) - (int(pre_date[0])*365 + int(pre_date[1])*30 + int(pre_date[2]))
            temp.append(days)
        else:
            if len(temp) == 0:
                all_days.append([-100])
            else:
                all_days.append(temp)
            temp = list()
        (pre_pro_id, pre_date), pre_q, pre_sub = all_tuples[loc]
    if len(temp) == 0:
        all_days.append([-100])
    else:
        all_days.append(temp)

    # print(all_days)
    avg_days = [0]*len(all_days)
    for loc, days in enumerate(all_days):
        avg_days[loc] = sum(days)/len(days)

    data = {'pro': list(set([x for x, y in list(df_grouped.index)])),
           'day': avg_days}
    req_df = pd.DataFrame(data)
    req_df = req_df[req_df['day']>=0]
    req_df = req_df[req_df['day']<=31]
    suggested_pro = req_df['pro'].tolist()
    trust_index = req_df['day'].tolist()

    return suggested_pro, trust_index

In [3]:
# 1123696, 1328656, 1330784, 1296512
suggested_pro, avg_result = get_suggestions(1296512)
suggested_pro
avg_result

[1902320]

[1.0]

In [ ]:
maxi = list()
mini = list()
entire_sugg = list()
entire_trust = list()
entire_ids = list()
count = 0
for i in df['customer_id'].tolist():
    count += 1
    suggested_pro, trust_index = get_suggestions(i)
    if len(suggested_pro) == 0:
        suggested_pro = ['a']
    if len(trust_index) == 0:
        trust_index = ['a']
    maxi.append(max(trust_index))
    mini.append(min(trust_index))
    entire_sugg.append(suggested_pro)
    entire_trust.append(trust_index)
    entire_ids.append(i)
    if count%1000==0:
        print("Till", count)

Till 1000
Till 2000
Till 3000
Till 4000
Till 5000
Till 6000
Till 7000
Till 8000
Till 9000
Till 10000
Till 11000
Till 12000
Till 13000
Till 14000
Till 15000
Till 16000
Till 17000
Till 18000
Till 19000
Till 20000
Till 21000
Till 22000
Till 23000
Till 24000
Till 25000
Till 26000
Till 27000
Till 28000
Till 29000
Till 30000
Till 31000
Till 32000
Till 33000
Till 34000
Till 35000
Till 36000
Till 37000
Till 38000
Till 39000
Till 40000
Till 41000
Till 42000
Till 43000
Till 44000
Till 45000
Till 46000
Till 47000
Till 48000
Till 49000
Till 50000
Till 51000
Till 52000
Till 53000
Till 54000
Till 55000
Till 56000
Till 57000
Till 58000
Till 59000
Till 60000
Till 61000
Till 62000
Till 63000
Till 64000
Till 65000
Till 66000
Till 67000
Till 68000
Till 69000
Till 70000
Till 71000
Till 72000
Till 73000
Till 74000
Till 75000
Till 76000
Till 77000
Till 78000
Till 79000
Till 80000
Till 81000
Till 82000
Till 83000
Till 84000
Till 85000
Till 86000
Till 87000
Till 88000
Till 89000
Till 90000
Till 91000
Till 920

#### Trials from here on

In [46]:
threshold = 3
df = df[df['product_quantity']>=threshold]
df = df[df['subscription']==0]
grouped_df1 = df.groupby(['customer_id', 'product_id','order_date'], sort=True).agg({'product_quantity': 'max', 'subscription': 'min'})
len(grouped_df1)
grouped_df1.head(51)

520933

product_quantity  subscription
customer_id product_id order_date                                
1121904     1124256    2019-06-16                 3             0
            1133216    2019-06-05                 4             0
            1414672    2019-06-22                 4             0
            1422960    2019-05-13                 4             0
1122016     1120112    2018-09-28                 3             0
            1688624    2018-05-04                 3             0
1122128     2166080    2019-05-05                 3             0
1122352     1553888    2019-06-08                 4             0
1123248     1120224    2019-03-16                 4             0
                       2019-03-29                 4             0
1123696     1120112    2019-07-24                 6             0
            2026640    2018-06-26                 3             0
                       2019-01-03                 3             0
1124144     1120112    2019-03-03                 4             0
            1120224    2018-09-08                 4             0
                       2018-09-12                 4             0
            1597792    2019-06-14                 6             0
            1645280    2019-06-03                 3             0
            1681904    2018-09-16                 3             0
            1688064    2019-04-09                 3             0
            1708336    2019-01-03                 4             0
            1709008    2019-04-08                 3             0
                       2019-04-09                 3             0
1124256     1120224    2018-08-05                 4             0
                       2018-08-06                 4             0
                       2018-08-08                 4             0
                       2018-08-10                 4             0
                       2018-08-11                 4             0
                       2018-08-12                 4             0
                       2018-08-13                 4             0
                       2018-08-16                 4             0
                       2018-08-18                 4             0
                       2018-08-19                 4             0
                       2018-08-21                 4             0
                       2018-08-22                 6             0
                       2018-08-23                 4             0
            1120560    2018-08-27                 4             0
                       2018-08-28                 4             0
                       2018-08-30                 6             0
                       2018-08-31                 4             0
                       2018-09-01                 4             0
                       2018-09-03                 6             0
                       2018-09-04                 4             0
                       2018-09-05                 5             0
                       2018-09-06                 4             0
                       2018-09-07                 4             0
                       2018-09-09                 6             0
                       2018-09-11                 4             0
                       2018-09-13                 4             0
                       2018-09-14                 4             0
                       2018-09-16                 6             0

In [5]:
i=0
all_tuples = list()
for tup in grouped_df1.itertuples():
    all_tuples.append(tuple(tup))
    
(pre_cust_id, pre_pro_id, pre_date), pre_qua, pre_sub = all_tuples[0]
all_days = list()
temp = list()
for loc in range(1, len(all_tuples)):
    (cust_id, pro_id, date), qua, sub = all_tuples[loc]
    #print(cust_id, pro_id, date)
    if pre_cust_id == cust_id and pre_pro_id == pro_id:
        pre_date = pre_date.split('-')
        date = date.split('-')
        days = (int(date[0])*365 + int(date[1])*30 + int(date[2])) - (int(pre_date[0])*365 + int(pre_date[1])*30 + int(pre_date[2]))
        temp.append(days)
    else:
        if len(temp) == 0:
            all_days.append([-100])
        else:
            all_days.append(temp)
        temp = list()
        
    (pre_cust_id, pre_pro_id, pre_date), pre_qua, pre_sub = all_tuples[loc]

if len(temp) == 0:
    all_days.append([-100])
else:
    all_days.append(temp)

In [11]:
avg_days = [0]*len(all_days)
for loc, days in enumerate(all_days):
    avg_days[loc] = sum(days)/len(days)

In [14]:
len(grouped_df1)
len(all_days)
len(avg_days)

520933

151635

151635

In [25]:
grouped_df1.head(25)

product_quantity  subscription
customer_id product_id order_date                                
1121904     1124256    2019-06-16                 3             0
            1133216    2019-06-05                 4             0
            1414672    2019-06-22                 4             0
            1422960    2019-05-13                 4             0
1122016     1120112    2018-09-28                 3             0
            1688624    2018-05-04                 3             0
1122128     2166080    2019-05-05                 3             0
1122352     1553888    2019-06-08                 4             0
1123248     1120224    2019-03-16                 4             0
                       2019-03-29                 4             0
1123696     1120112    2019-07-24                 6             0
            2026640    2018-06-26                 3             0
                       2019-01-03                 3             0
1124144     1120112    2019-03-03                 4             0
            1120224    2018-09-08                 4             0
                       2018-09-12                 4             0
            1597792    2019-06-14                 6             0
            1645280    2019-06-03                 3             0
            1681904    2018-09-16                 3             0
            1688064    2019-04-09                 3             0
            1708336    2019-01-03                 4             0
            1709008    2019-04-08                 3             0
                       2019-04-09                 3             0
1124256     1120224    2018-08-05                 4             0
                       2018-08-06                 4             0

threshold = 3
#df = df[df['product_quantity']>=threshold]
#df = df[df['subscription']==0]
grouped_df2 = df.groupby(['customer_id', 'product_id'], sort=True).agg({'product_quantity': 'max', 'subscription': 'min', 'order_date':'count'})
len(grouped_df2)
#grouped_df2.head(15)
freq_count = grouped_df2['order_date'].tolist()


new_avg_days = list()
for i, j in zip(avg_days, freq_count):
    new_avg_days += [i]*j
len(new_avg_days)

In [34]:
len(new_avg_days)
len(grouped_df1)

521971

520933

In [51]:
k = all_days
count = 0
for loc, data in enumerate(k):
    if len(data) == 1 and (-100) in data:
        count += 1
    else:
        count += len(data)+1
count

520933

In [88]:
len(all_days)

151635

In [90]:
new_avg_days = [0]*len(all_days)
k = all_days
count = 0
for loc, data in enumerate(k):
    if len(data) == 1 and (-100) in data:
        count += 1
        new_avg_days[loc] = [-100]
    else:
        count += len(data)+1
        new_avg_days[loc] = data+[0]

count
len(new_avg_days)
len(grouped_df1)

520933

151635

520933

In [91]:
from itertools import chain
trust_index = list(chain(*new_avg_days))
len(trust_index)
trust_index[:10]

520933

[-100, -100, -100, -100, -100, -100, -100, -100, 13, 0]

In [92]:
grouped_df1['trust_index'] = trust_index
grouped_df1.head()

product_quantity  subscription  trust_index
customer_id product_id order_date                                             
1121904     1124256    2019-06-16                 3             0         -100
            1133216    2019-06-05                 4             0         -100
            1414672    2019-06-22                 4             0         -100
            1422960    2019-05-13                 4             0         -100
1122016     1120112    2018-09-28                 3             0         -100

In [96]:
final_df = grouped_df1[grouped_df1['trust_index']>-100]
final_df = final_df[grouped_df1['trust_index']<=31]
len(final_df)
final_df.head()

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


389153

product_quantity  subscription  trust_index
customer_id product_id order_date                                             
1123248     1120224    2019-03-16                 4             0           13
                       2019-03-29                 4             0            0
1123696     2026640    2019-01-03                 3             0            0
1124144     1120224    2018-09-08                 4             0            4
                       2018-09-12                 4             0            0

In [105]:
final_df.to_csv("processed_content.csv", index=False)

In [ ]:
"1540448": {
            "product_suggestions": [1210, 1234, 1235],
            "trust_index": [-29.5, 29, 1000]
        },

In [109]:
len(set(entire_cust))

21503

In [116]:
#result_df = df.loc[(df.index.get_level_values('A') > 1.7) & (df.index.get_level_values('B') < 666)]
entire_cust = list(final_df.index.get_level_values('customer_id'))
entire_pro = list(final_df.index.get_level_values('product_id'))

final_dict = dict()
for i, j, k in zip(entire_cust, entire_pro, entire_trust):
    try:
        final_dict[str(i)]["product_suggestions"] += [str(j)]
        final_dict[str(i)]["discount_trust_index"] += [str(k/3)]
    except:
        final_dict[str(i)] = dict()
        final_dict[str(i)]["product_suggestions"] = list()
        final_dict[str(i)]["product_suggestions"] += [str(j)]
        final_dict[str(i)]["discount_trust_index"] = list()
        final_dict[str(i)]["discount_trust_index"] += [str(k/3)]

In [194]:
import json
import json
with open('sub_suggestions.json', 'r') as file:
    data = json.load(file)

keys = list(data.keys())
new_data = dict()
for key in keys:
    new_data[key] = {"product_suggestions": list(set(data[key]["product_suggestions"])),
                "discount_trust_index": list(set(data[key]["discount_trust_index"]))
                }

In [195]:
import json

with open('sub_suggestions_2.json', 'w') as file:
    json.dump(new_data, file)

#### 2nd Trials

In [5]:
cust = list(grouped_df1.index.get_level_values('customer_id'))
pro = list(grouped_df1.index.get_level_values('product_id'))
cust_pro = dict()
for i, j in zip(cust, pro):
    try:
        cust_pro[str(i)] += [str(j)]
    except:
        cust_pro[str(i)] = []
        cust_pro[str(i)] += [str(j)]

In [12]:
import json

with open('subscription_eligibility.json', 'w') as file:
    json.dump(cust_pro, file)

In [59]:
group_3 = df.groupby(['customer_id', 'product_id', 'order_date'], sort=True).agg({'product_quantity': 'max', 'subscription': 'min'})
group_4 = group_3[group_3['subscription']==0]
group_4.head(15)

product_quantity  subscription
customer_id product_id order_date                                
1121904     1120224    2019-05-08                 1             0
                       2019-05-09                 1             0
                       2019-05-10                 1             0
                       2019-05-11                 1             0
                       2019-05-13                 1             0
                       2019-05-15                 1             0
                       2019-05-16                 1             0
                       2019-05-17                 1             0
                       2019-06-01                 1             0
            1120336    2019-05-19                 1             0
                       2019-05-20                 1             0
                       2019-05-21                 1             0
                       2019-05-23                 1             0
                       2019-05-24                 1             0
                       2019-05-25                 2             0

In [60]:
i=0
all_tuples = list()
for tup in group_4.itertuples():
    all_tuples.append(tuple(tup))

In [61]:
all_tuples[:15]

[((1121904, 1120224, '2019-05-08'), 1, 0),
 ((1121904, 1120224, '2019-05-09'), 1, 0),
 ((1121904, 1120224, '2019-05-10'), 1, 0),
 ((1121904, 1120224, '2019-05-11'), 1, 0),
 ((1121904, 1120224, '2019-05-13'), 1, 0),
 ((1121904, 1120224, '2019-05-15'), 1, 0),
 ((1121904, 1120224, '2019-05-16'), 1, 0),
 ((1121904, 1120224, '2019-05-17'), 1, 0),
 ((1121904, 1120224, '2019-06-01'), 1, 0),
 ((1121904, 1120336, '2019-05-19'), 1, 0),
 ((1121904, 1120336, '2019-05-20'), 1, 0),
 ((1121904, 1120336, '2019-05-21'), 1, 0),
 ((1121904, 1120336, '2019-05-23'), 1, 0),
 ((1121904, 1120336, '2019-05-24'), 1, 0),
 ((1121904, 1120336, '2019-05-25'), 2, 0)]

In [75]:
(pre_cust_id, pre_pro_id, pre_date), pre_pro, pre_sub = ((1121904, 1120224, '2019-05-08'), 1, 0)
all_days = list()
temp = list()
for loc in range(1, len(all_tuples)):
    (cust_id, pro_id, date), pro, sub = all_tuples[loc]
    #print(cust_id, pro_id, date)
    if pre_cust_id == cust_id and pre_pro_id == pro_id:
        pre_date = pre_date.split('-')
        date = date.split('-')
        days = (int(date[1])*30 + int(date[2])) - (int(pre_date[1])*30 + int(pre_date[2]))
        temp.append(days)
    else:
        all_days.append(temp)
        temp = list()
    (pre_cust_id, pre_pro_id, pre_date), pre_pro, pre_sub = all_tuples[loc]

In [86]:
avg_days = [0]*2796619
for loc, days in enumerate(all_days):
    if len(days) == 0:
        avg_days[loc] = 1000
    else:
        avg_days[loc] = sum(days)/len(days)

In [87]:
avg_days[:10]

[2.875, 3.0, 1000, 6.5, 13.0, 1000, 1000, 1000, 1000, 1000]

In [98]:
df_edit = df[df['subscription']==0]
group_6 = df_edit.groupby(['customer_id', 'product_id'], sort=True).agg({'product_quantity': 'max', 'subscription': 'min', 'order_date': 'count'})


product_quantity  subscription  order_date
customer_id product_id                                            
1121904     1120224                    1             0           9
            1120336                    2             0          16
            1120672                    1             0           1
            1120896                    1             0          11
            1124256                    3             0           5
            1133216                    4             0           1
            1360800                    1             0           1
            1367072                    1             0           1
            1379840                    1             0           1
            1414672                    4             0           1
            1422960                    4             0           1
            1518832                    1             0           1
            1569792                    1             0           1
            1579648                    1             0           1
            1645840                    1             0           1

In [101]:
freq_count = group_6['order_date'].tolist()
freq_count[:10]

[9, 16, 1, 11, 5, 1, 1, 1, 1, 1]

In [112]:
new_avg_days = list()
for i, j in zip(avg_days, freq_count):
    new_avg_days += [i]*j
new_avg_days[:10]

[2.875, 2.875, 2.875, 2.875, 2.875, 2.875, 2.875, 2.875, 2.875, 3.0]

In [113]:
#group_4['avg_cycle'] = new_avg_days
len(group_4)

7396563

In [114]:
len(new_avg_days)

7399498

In [124]:
a = group_4.head(7396563)
a['avg_cycle'] = new_avg_days[:7396563]
final_dataframe = a.head(7396563)

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [125]:
final_dataframe.head()

product_quantity  subscription  avg_cycle
customer_id product_id order_date                                           
1121904     1120224    2019-05-08                 1             0      2.875
                       2019-05-09                 1             0      2.875
                       2019-05-10                 1             0      2.875
                       2019-05-11                 1             0      2.875
                       2019-05-13                 1             0      2.875

In [131]:
a = final_dataframe['avg_cycle'].tolist()
count = 0
for i in a: 
    if 0 < i < 31:
        count+=1
count

3358978

In [153]:
threshold = 5
req_df = final_dataframe[final_dataframe['product_quantity']>=threshold]

In [193]:
cust1 = list(req_df.index.get_level_values('customer_id'))
pro1 = list(req_df.index.get_level_values('product_id'))
avg = req_df['avg_cycle'].tolist()
cust_pro1 = dict()
for i, j, k in zip(cust1, pro1, avg):
    try:
        if str(j) in cust_pro1[str(i)]:
            continue
        cust_pro1[str(i)] += [str(j)]
        cust_pro1[str(i)+"t"] = str(k)
    except:
        cust_pro1[str(i)] = []
        cust_pro1[str(i)] += [str(j)]
        cust_pro1[str(i)+"t"] = str(k)
        
import json

with open('subscription_eligibility_time3.json', 'w') as file:
    json.dump(cust_pro1, file)

In [191]:
len(cust1)

106229

In [192]:
len()

106229

In [168]:
new_list = list()
for loc, data in enumerate(list(cust_pro1.values())):
    new_list.append(list(set(data)))
len(new_list)
len(list(cust_pro1.values()))

34032

34032

In [180]:
dict_data = dict()
for i, j in zip(cust1, new_list):
    dict_data[str(i)] = j
dict_data

{'1123696': ['1120112'],
 '1124144': ['1597792'],
 '1124256': ['1120224', '1668240', '1120560'],
 '1128512': ['1120224', '1120560'],
 '1129072': ['2412928',
  '3136672',
  '1432480',
  '1649872',
  '2210208',
  '1636320',
  '1432704',
  '1569456',
  '1417696'],
 '1129296': ['1120448'],
 '1134000': ['1639120'],
 '1136576': ['2333408'],
 '1139376': ['2026640', '2181760'],
 '1139824': ['1415344', '1414672', '1417136', '1640352', '1574048'],
 '1139936': ['1120112'],
 '1140160': ['2165968', '1120112', '2240336'],
 '1140608': ['1597904', '2375744', '1984864', '1709008', '1120224', '1645728'],
 '1143520': ['2166080', '2310784'],
 '1145536': ['1125600'],
 '1145872': ['1422848',
  '1916208',
  '3105984',
  '1125600',
  '1422960',
  '1144080',
  '1553888',
  '1645728'],
 '1147664': ['2106272', '1125600'],
 '1150912': ['1661408', '2570736', '1125600'],
 '1151360': ['1120560'],
 '1153824': ['1120224', '1666896', '1673504'],
 '1155168': ['2333408',
  '1371552',
  '2336208',
  '2825536',
  '1870400'

In [182]:
cust1[:3]

[1123696, 1124144, 1124256]

In [183]:
new_list[:3]

[['1120112'],
 ['1597792'],
 ['1570912', '1120560', '1125600', '1120224', '1645728', '1964368']]

In [188]:
dat = dict()
for i,j in zip(cust1, new_list):
    dat[str(i)] = j


In [189]:
cust1

[1123696,
 1124144,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1124256,
 1128512,
 1128512,
 1128512,
 1129072,
 1129296,
 1129296,
 1129296,
 1134000,
 1134000,
 1134000,
 1136576,
 1139376,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139824,
 1139936,
 1139936,
 1139936,
 1140160,
 1140160,
 1140160,
 1140160,
 1140608,
 1140608,
 1140608,
 1140608,
 1140608,
 1140608,
 1140608,
 1143520,
 1145536,
 1145536,
 1145872,
 1147664,
 1150912,
 1150912,
 1150912,
 1150912,
 1150912,
 1150912,
 1150912,
 1151360,
 1151360,
 1151360,
 1153824,
 1155168,
 1155168,
 1155168,
 1155168,
 1155168,
 1155168,
 1155168,


#### Already subscribed people

In [118]:
import time
st = time.time()
df = pd.read_csv('temp_bq_data/hackathon_data.csv')
df.head()
print("took", time.time()-st)
#df.info()

,customer_id,manufacturer_id,society_id,city_id,route_id,store_id,order_id,order_date,category_id,subcategory_id,product_id,product_quantity,selling_price_per_unit,total_cost,subscription,product_addedtobasket_on
0,2698080,1122016,1127168,1120112,1121456,1120112,338048928,2018-05-01,1122576,1125264,1120336,4,16.01,64.04,0,2018-04-30 16:23:46
1,1134224,1134336,1120224,1120112,1120336,1120112,337533168,2018-05-01,1122576,1125152,1939280,1,8.79,8.79,1,2018-04-19 01:30:02
2,3686704,1150128,1126160,1120112,1123472,1120112,338235520,2018-05-01,1123472,1130640,1681456,1,25.00,25.00,0,2018-04-30 22:26:07
3,4045328,1122016,1137360,1120112,1121456,1120112,337818768,2018-05-01,1122576,1125264,1120224,3,20.38,61.14,1,2018-04-23 01:30:03
4,3568656,1122016,1120784,1120112,1122576,1120112,338234400,2018-05-01,1122576,1125264,1120112,3,25.38,76.14,0,2018-04-30 22:22:57


took 19.55033802986145


In [120]:
len(df)
df = df[df['subscription']==1]
len(df)

9678979

2279480

In [121]:
group_df = df.groupby(['customer_id', 'product_id']).agg({'subscription':'min'})
group_df.head()

subscription
customer_id product_id              
1121904     1120224                1
            1120336                1
            2166080                1
1122016     1120112                1
            1120224                1

In [125]:
cust = list(group_df.index.get_level_values('customer_id'))
pro = list(group_df.index.get_level_values('product_id'))


In [134]:
len(pro)

44593

In [135]:
final_dict1 = dict()
for i, j in zip(cust, pro):
    try:
        final_dict1[str(i)]["subscribed_products"] += [str(j)]
    except:
        final_dict1[str(i)] = dict()
        final_dict1[str(i)]["subscribed_products"] = list()
        final_dict1[str(i)]["subscribed_products"] += [str(j)]

In [136]:
final_dict1

{'1121904': {'subscribed_products': ['1120224', '1120336', '2166080']},
 '1122016': {'subscribed_products': ['1120112', '1120224']},
 '1122128': {'subscribed_products': ['1120336']},
 '1122352': {'subscribed_products': ['1120224', '1570912', '2049376']},
 '1123248': {'subscribed_products': ['1120224']},
 '1123696': {'subscribed_products': ['1120224', '2026640']},
 '1124144': {'subscribed_products': ['1120224']},
 '1124256': {'subscribed_products': ['1120224',
   '1120672',
   '1570912',
   '2356816']},
 '1124592': {'subscribed_products': ['1120224']},
 '1126608': {'subscribed_products': ['1120112', '1120224']},
 '1128400': {'subscribed_products': ['1120112', '1120336']},
 '1128512': {'subscribed_products': ['1120224', '1120672']},
 '1129296': {'subscribed_products': ['1120336', '1120448', '1570912']},
 '1133888': {'subscribed_products': ['1120112',
   '1120224',
   '1121120',
   '1124256',
   '1125936']},
 '1134000': {'subscribed_products': ['1120112', '1120224']},
 '1134224': {'subscr

In [137]:
with open('subscribed_cust.json', 'w') as file:
    json.dump(final_dict1, file)

In [140]:
df_grouped = df.groupby(['customer_id', 'product_id']).agg({'product_quantity':'sum'})

In [158]:
g = df_grouped['product_quantity'].groupby(level=0, group_keys=False)

In [177]:
z[:10]

customer_id  product_id
1121904      1120336        35
             2166080        27
             1120224         8
1122016      1120224       253
             1120112        95
1122128      1120336       249
1122352      1120224        90
             2049376        87
             1570912        86
1123248      1120224       511
Name: product_quantity, dtype: int64

In [176]:
z = g.nlargest(3)

In [182]:
x = pd.DataFrame(z)
x.head()

product_quantity
customer_id product_id                  
1121904     1120336                   35
            2166080                   27
            1120224                    8
1122016     1120224                  253
            1120112                   95

In [173]:
import pandas as pd
import numpy as np
data = np.array(['a','b','c','d','e','f'])
s = pd.Series(data,index=[100,101,102,103,104,105])
 
s
# retrieve multiple elements with labels or index
print(s[[102,103,104]])


100    a
101    b
102    c
103    d
104    e
105    f
dtype: object

102    c
103    d
104    e
dtype: object


In [143]:
df_grouped.tail(20)

product_quantity
customer_id product_id                  
47013904    1674736                    8
            2026640                   63
            3621632                   31
47044368    1121008                   10
            1858864                    2
47046496    1915760                    4
47090064    1120224                   10
            1645280                    3
            1670032                   10
            1741936                    1
            1862448                    1
            1939280                    4
            2144576                    2
            2166080                    7
            2468816                   10
47358080    1120560                   85
            1668240                   14
            1707888                   69
48171648    1698256                   45
            1939280                    1

In [147]:
a = df_grouped.reset_index(level=['customer_id', 'product_id'])
a.head()

,customer_id,product_id,product_quantity
0,1121904,1120224,8
1,1121904,1120336,35
2,1121904,2166080,27
3,1122016,1120112,95
4,1122016,1120224,253


In [157]:
b = a.sort_values(by=['product_quantity', 'product_id','customer_id'], ascending=False)
b.head(100)

,customer_id,product_id,product_quantity
6305,5560240,1120224,3909
2301,2772112,1120224,3420
2279,2756768,1120336,3328
6376,5619264,1120336,3141
99,1163792,1120224,2778
5137,4757312,1120224,2751
297,1321264,1120224,2681
790,1681232,1120224,2675
289,1304352,1120560,2562
2767,3130624,2026640,2529


In [186]:
y = x.reset_index(level=['customer_id', 'product_id'])
y.head()

,customer_id,product_id,product_quantity
0,1121904,1120336,35
1,1121904,2166080,27
2,1121904,1120224,8
3,1122016,1120224,253
4,1122016,1120112,95


In [189]:
cust = y['customer_id'].tolist()
pro = y['product_id'].tolist()
# z[[(1121904, 1120336)]]

final_dict2 = dict()
for i, j in zip(cust, pro):
    try:
        final_dict2[str(i)] += [str(j)]
    except:
        final_dict2[str(i)] = list()
        final_dict2[str(i)] = [str(j)]

In [191]:
with open('top_3.json', 'w') as file:
    json.dump(final_dict2, file)

In [ ]:
product_suggestions